# import libs

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from GCN import StaticGCN as GCN


# import data

In [ ]:
# paths pre-setting
label_root = f'/home/{os.getlogin()}/ERIE/silicone/output'
model_root = f'/home/{os.getlogin()}/ERIE/silicone/Track-Shuyuan-2023-08-13/videos'
model_name = 'DLC_resnet50_TrackAug13shuffle1_50000'


In [ ]:
# pre setting edges sets
edges = [(0,1),(0,2),(1,3),(2,3),(3,4),(3,5),(4,6),(5,7),
         (8,9),(8,10),(9,11),(10,11),(11,12),(11,13),(12,14),(13,15)]
onehot_matrix = np.eye(16)

edge_index = np.zeros((2, len(edges)), dtype=np.int64)

for i, edge in enumerate(edges):
    edge_index[:, i] = edge


In [ ]:
# pre setting training sets
training_sets = ['C_M1_T1_1', 'C_M1_T1_3', 'C_M1_T1_5', 'C_M1_T1_7',
                 'R2_M1_T1_1', 'R2_M1_T1_3', 'R2_M1_T1_5', 'R2_M1_T1_7',
                 'L2_M1_T1_1', 'L2_M1_T1_3', 'L2_M1_T1_5', 'L2_M1_T1_7',
                 'Z2_M1_T1_1', 'Z2_M1_T1_3', 'Z2_M1_T1_5']

# initial arrays
X_train = np.zeros((0, 32))
y_train = np.zeros((0, 3))

for set in tqdm(training_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    coordinates_L = coordinates_L.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    coordinates_R = coordinates_R.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    labels = labels[:frames, 7:10]
    coordinates = np.hstack((coordinates_L, coordinates_R))

    X_train = np.vstack((X_train, MinMaxScaler().fit_transform(coordinates)))
    y_train = np.vstack((y_train, MinMaxScaler().fit_transform(labels)))

feature_matrices = []
for points in X_train.reshape(-1, 16, 2):
    feature_matrices.append(np.hstack((onehot_matrix, points)))

X_train = np.array(feature_matrices)


In [ ]:
X_train.shape, edge_index.shape, y_train.shape

# Trainning and validation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(input_dim=X_train.shape[2], hidden_dim=64, output_dim=y_train.shape[1],
            device=device, batch_size=32, l2_reg=0.0001, lr=0.0001, random_seed=42)
model.train(X_train, edge_index, y_train,
            epochs=100, use_tqdm=True, save_loss=True)


# Retrieve model

In [ ]:
net_name = input()

In [ ]:
import json

with open(f'results/losses_{net_name}.json', 'r') as f:
    params = json.load(f)

input_dim = params["input_dim"]
hidden_dim = params["hidden_dim"]
output_dim = params["output_dim"]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# weights = params["weights"]
# num_hidden_layers = params["num_hidden_layers"]
lr = params["lr"]
batch_size = params["batch_size"]
l2_reg = params["l2_reg"]
random_seed = params["random_seed"]

model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, 
            device=device, batch_size=batch_size, l2_reg=l2_reg, 
            lr=lr, random_seed=random_seed)


In [ ]:
model.load_state_dict(torch.load(
    f'models/best_model_{net_name}.pth', map_location=device))


# Test

In [ ]:
# pre setting test sets
test_sets = ['C_M1_T1_8', 'M1_NF', 'M2_NF', 'R1_M1_T1_1',
             'R1_M1_T1_2', 'R3_M1_T1_1', 'R3_M1_T1_2', 'L1_M1_T1_1',
             'L1_M1_T1_2', 'L3_M1_T1_1', 'L3_M1_T1_2', 'Z1_M1_T1_1',
             'Z3_M1_T1_1', 'R2_M1_T1_8', 'M3_R2_NF', 'M5_R2_NF',
             'M5_L2_NF', 'M3_L2_NF', 'M7_L2_NF', 'Z2_M1_T1_5',
             'M9_Z2_NF', 'M8_Z2_NF']

rmse = np.zeros((3, len(test_sets)))

for i, set in enumerate(test_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    coordinates_L = coordinates_L.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    coordinates_R = coordinates_R.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    labels = labels[:frames, 7:10]
    coordinates = np.hstack((coordinates_L, coordinates_R))

    X_test = MinMaxScaler().fit_transform(coordinates)
    y_test = MinMaxScaler().fit_transform(labels)

    feature_matrices = []
    for points in X_test.reshape(-1, 16, 2):
        feature_matrices.append(np.hstack((onehot_matrix, points)))
    X_test = np.array(feature_matrices)

    y_pred = model.predict(X_test, edge_index, y_test)

    from sklearn.metrics import mean_squared_error
    for axis in range(3):
        rmse[axis, i] = mean_squared_error(y_test[:, axis], y_pred[:, axis], squared=False)

    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    fig.suptitle(f"Position - Time plot of imageset {set}")
    titles = ['X', 'Y', 'X']
    fps = 30  # Frames per second
    time_values = [i/fps for i in range(y_test.shape[0])]
    for i in range(3):
        ax[i].plot(time_values, y_test[:, i], color='black', label='actual')
        ax[i].plot(time_values, y_pred[:, i], color='red', label='predict')
        if i == 1:
            ax[i].legend()
        ax[i].set_ylabel(f'norm position {titles[i]}')
        ax[i].set_xlim([0, np.max(time_values)])
        ax[i].set_ylim([-0.1, 1.1])

    # Set the x-label only for the bottom subplot
    ax[-1].set_xlabel('Time (s)')

    plt.tight_layout()
    plt.savefig(os.path.join(os.getcwd(), f'plots/{set}.pdf'), format='pdf')
    plt.show()


In [ ]:
# Calculate mean and standard deviation of RMSE for all dimensions
mean_rmse = np.mean(rmse, axis=1)
std_rmse = np.std(rmse, axis=1)

# Print mean and std for all dimensions
print("Average RMSE (mean ± std):")
print(f"{np.mean(mean_rmse):.3f} ± {np.mean(std_rmse):.3f}")

# Calculate mean and standard deviation of RMSE for each dimension and model
dimensions = ['x', 'y', 'z']
print("\nRMSE (mean ± std) for each dimension:")
for i, dim in enumerate(dimensions):
    print(f"{dim}: {mean_rmse[i]:.3f} ± {std_rmse[i]:.3f}")


# Plots

In [ ]:
import json
import matplotlib.pyplot as plt

with open('results/losses_20230808_002115.json', 'r') as f:
    data = json.load(f)

losses = data['losses']

plt.figure(figsize=(10, 6))
plt.plot(range(len(losses)), losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GCN Training Loss over Epochs')
plt.grid(True)
plt.show()